# DEDS Werkcollege 2.2

## Imports

In [2]:
import pyodbc
import sqlite3 as sql
import pandas as pd

## Connectie SSMS

In [3]:
DB = {
    "servername" : r"VISSIE\SQLEXPRESS",
    "database" : "SDM"}

export_conn = pyodbc.connect(f"""DRIVER={'SQL SERVER'};
                             SERVER={DB['servername']};
                             DATABASE={DB['database']};
                             Trusted_Connection=yes
                             """)

export_conn.setencoding('utf-8')
export_conn.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
export_conn.setdecoding(pyodbc.SQL_WCHAR, encoding='utf-8')

export_cursor = export_conn.cursor()

## Connecties SQL Databases (raw)

In [4]:
connection_crm = sql.connect(r"..\..\..\data\raw\go_crm_train.sqlite")
connection_sales = sql.connect(r"..\..\..\data\raw\go_sales_train.sqlite")
connection_staff = sql.connect(r"..\..\..\data\raw\go_staff_train.sqlite")

## Dataframes
alle tabelnamen worden opgehaald door de create_dataframes_sql functie. Alle gelezen dataframes worden in een dictionary gestopt, met als key de tabel naam.

In [5]:
def create_dataframes_sql(connection, db_type : str):
    dictionary : dict = {}
    query : str = ""
    key : str = ""
    
    if (db_type == "sqlite"):
        query = "SELECT name FROM sqlite_master WHERE type='table';"
        key = "name"
    elif (db_type == "ssms"):
        query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';"
        key = "TABLE_NAME"
    else:
        return {}
    table_names = pd.read_sql(query, connection)
    
    for table in table_names[key].tolist():
        dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)
    
    return dictionary

def create_dataframes_csv(path, name):
    frame = pd.read_csv(path)
    
    if 'Unnamed: 0' in frame.columns:
        frame = frame.drop('Unnamed: 0', axis=1)
    
    return {name : frame}

    

crm = create_dataframes_sql(connection_crm, "sqlite")
sales = create_dataframes_sql(connection_sales, "sqlite")
staff = create_dataframes_sql(connection_staff, "sqlite")
inventory = create_dataframes_csv(r"..\..\..\data\raw\inventory_levels_train.csv", "inventory_levels")
forecast = create_dataframes_csv(r"..\..\..\data\raw\product_forecast_train.csv", "forecast")
sdm = create_dataframes_sql(export_conn, "ssms")

C:\Users\jdvis_x5odeao\AppData\Local\Temp\ipykernel_68424\3080538893.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table_names = pd.read_sql(query, connection)
C:\Users\jdvis_x5odeao\AppData\Local\Temp\ipykernel_68424\3080538893.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)


## Query runners

In [6]:
from typing import Iterable

def query_remove(table_name : str):
    try:
        query = f"DELETE FROM {table_name}"
        export_cursor.execute(query)
        export_cursor.commit()
    except pyodbc.Error as e:
        print(f"ERROR: {table_name}: {e}")
        return table_name
    
    return None

def rigth_type(value, column_name, types):
    dtype = types[column_name]
    
    if pd.isna(value):
        return "NULL"
    
    if dtype == "object" or dtype == "string":
        value = value.replace("'", "''")
        return f"'{value}'"
    
    return f"{value}"
    

def create_add_query(row, types):
    query = ""
    columns = list(row.keys())
    
    for pos in range(len(columns)):
        column_name = columns[pos]
        value = row[column_name]
        
        if (pos == len(columns) - 1):
            query += f"{column_name}) VALUES ("
        else:
            query += f"{column_name}, "
    
    data = list(row)
    
    for pos in range(len(data)):
        column_name = columns[pos]
        value = row[column_name]
        
        if pos == len(columns) - 1:
            query += f"{rigth_type(value, column_name, types)})"
        else:
            query += f"{rigth_type(value, column_name, types)}, "
    
    return query

def query_add(table_name : str, table_data : pd.DataFrame):
    queries = []
    types = table_data.dtypes

    for index, row in table_data.iterrows():
        query : str = f"INSERT INTO {table_name} ("
        query += create_add_query(row, types)
        queries.append(query)
    
    return queries
        

## Leegmaken SDM Database

In [7]:
def empty_sdm():
    tables_sdm : list = [
        'inventory_levels',
        'forecast', 
        'sales_demographic', 
        'retailer', 
        'retailer_type', 
        'retailer_headquarters', 
        'retailer_segment', 
        'age_group', 
        'training', 
        'satisfaction', 
        'satisfaction_type', 
        'course', 
        'returned_item', 
        'return_reason', 
        'order_details', 
        'product', 
        'product_type', 
        'product_line', 
        'order_header', 
        'retailer_contact', 
        'sales_staff', 
        'sales_branch', 
        'retailer_site', 
        'order_method', 
        'country', 
        'sales_territory'
    ]

    while len(tables_sdm) > 0:
        for table in tables_sdm:
            try:
                temp = query_remove(table)
                
                if (temp == None):
                    tables_sdm.remove(table)
                    print(f"REMOVED {table}")
                else:
                    print(f"NOT YET REMOVED: {table}")
                    
            except pyodbc.Error as e:
                print(e)

    print("All items are removed")

empty_sdm()

REMOVED inventory_levels
REMOVED sales_demographic
ERROR: retailer_type: ('23000', '[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The DELETE statement conflicted with the REFERENCE constraint "FK__retailer__RETAIL__08B54D69". The conflict occurred in database "SDM", table "dbo.retailer", column \'RETAILER_TYPE_CODE\'. (547) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)')
NOT YET REMOVED: retailer_type
ERROR: retailer_headquarters: ('23000', '[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The DELETE statement conflicted with the REFERENCE constraint "FK__retailer__RETAIL__07C12930". The conflict occurred in database "SDM", table "dbo.retailer", column \'RETAILER_CODEMR\'. (547) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)')
NOT YET REMOVED: retailer_headquarters
ERROR: retailer_segment: ('23000', '[23000] [Microsoft][ODBC SQL Server 

## Toevoegen data SDM Database
Inventory_levels en Forecast worden wel toegevoegd, maar gegeven terug dat er niks gebeurt wegens een gedupliceerde sleutel

In [8]:
def remove_duplicate_columns(end_frame : pd.DataFrame, merge_frame : pd.DataFrame):
    for column in merge_frame.columns:
        if column in end_frame.columns:
            merge_frame.drop(column, axis=1, errors='ignore')
    
    return merge_frame

def merge_all_in_one(end_frame: dict, merge_frame: dict):
    end_frame = end_frame | merge_frame
    
    return end_frame

sdm_keys = list(sdm.keys())

empty_sdm()
    
sdm : dict = {}
sdm = merge_all_in_one(sdm, crm)
sdm = merge_all_in_one(sdm, sales)
sdm = merge_all_in_one(sdm, staff)
sdm = merge_all_in_one(sdm, forecast)
sdm = merge_all_in_one(sdm, inventory)
sdm['country'] = sdm["country"].rename(columns={"COUNTRY_EN" : "COUNTRY"})
sdm['country']['FLAG_IMAGE'] = crm['country']['FLAG_IMAGE']
sdm['country']['SALES_TERRITORY_CODE'] = crm['country']['SALES_TERRITORY_CODE']
    
sdm_keys_sorted : list = [
    'sales_territory', 
    'country', 
    'order_method', 
    'retailer_site', 
    'sales_branch', 
    'sales_staff', 
    'retailer_contact', 
    'order_header',
    'product_line', 
    'product_type', 
    'product',
    'order_details', 
    'return_reason', 
    'returned_item', 
    'course', 
    'satisfaction_type', 
    'satisfaction',
    'training',
    'age_group',
    'retailer_segment',
    'retailer_headquarters',
    'retailer_type',
    'retailer',
    'sales_demographic',
    'inventory_levels',
    'forecast'
]

sdm_keys_done : list = []

queries = []

for key in sdm_keys_sorted:
    for query in query_add(key, sdm[key]):
        queries.append(query)
        try:
            export_cursor.execute(query)
            queries.remove(query)
        except pyodbc.Error as e:
            print(query)
            print(e)
            break
    
    sdm_keys_done.append(key)

export_cursor.commit()

    

REMOVED inventory_levels
REMOVED sales_demographic
REMOVED retailer_type
REMOVED retailer_segment
REMOVED training
REMOVED satisfaction_type
REMOVED returned_item
REMOVED order_details
REMOVED product_type
REMOVED order_header
REMOVED sales_staff
REMOVED retailer_site
REMOVED country
REMOVED forecast
REMOVED retailer_headquarters
REMOVED satisfaction
REMOVED return_reason
REMOVED product_line
REMOVED sales_branch
REMOVED sales_territory
REMOVED retailer
REMOVED course
REMOVED retailer_contact
REMOVED age_group
REMOVED order_method
REMOVED product
All items are removed
INSERT INTO inventory_levels (INVENTORY_YEAR, INVENTORY_MONTH, PRODUCT_NUMBER, INVENTORY_COUNT) VALUES (2023, 5, 48, 1978)
('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK__inventor__E115999194B81EA5'. Cannot insert duplicate key in object 'dbo.inventory_levels'. The duplicate key value is (48). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQ